In [1]:
# Forked from https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d/notebook

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os #mkdir, path.exists..

In [3]:
#https://docs.python.org/3/library/warnings.html
import warnings
#don't print messages of future warings
warnings.filterwarnings(action="ignore",category=FutureWarning)

In [4]:
#Constants

#File paths
INPUT_DIR="./input"
TRAIN_PATH=INPUT_DIR+"/train.json"
TEST_PATH=INPUT_DIR+"/test.json"

MODEL_DIR="./models"

In [5]:
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        print("{} doesn't exist. Creating...".format(directory))
        os.mkdir(directory)
        
create_directory_if_not_exists(INPUT_DIR)
create_directory_if_not_exists(MODEL_DIR)

In [6]:
def prepare_data(data):
    '''data - train/test data as a pandas DataFrame'''
    #Create 3 bands having HH, HV and avg of both
    X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_1"]])
    X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_2"]])
    result = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
    return result

In [7]:
#Load the data. NOTE: make sure you have downloaded and unziped the files
train = pd.read_json(TRAIN_PATH)

#transform it in the right format for training, applying data augmentation etc
X_train = prepare_data(train)

In [8]:
#Keras imports
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


In [9]:
#define our model
def getModel():
    #Building the model
    model=Sequential()
    #Conv Layer 1
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(75,75,3),padding="same"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

    #Conv Layer 2
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #Conv Layer 3
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu' , padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #Conv Layer 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #Conv Layer 5
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #Conv Layer 6
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    #Flatten the data for upcoming dense layers
    model.add(Flatten())

    #Dense Layers
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    #Dense Layer 2
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    #Sigmoid Layer
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath, patience=2):
    '''Returns an array of callbacks which will be called during the training of the model.'''
    #stop training if there were no improvements after `patience` epochs
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    
    #save the model after every epoch.
    #save_best_only - do not override if the loss was higher than in the previous epoch.
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [10]:
def get_model_from_file_or_create(file_path):
    model=getModel()
    #try:
    #    model.load_weights(file_path)
    #except OSError:
    #    print("Could not load model from file: {}.".format(file_path))
    return model

In [11]:
from sklearn.model_selection import train_test_split
target_train=train['is_iceberg']
#split into train and validation data using 75% of the data as train data
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

In [12]:
#Keras uses the HDF5 format 
file_path = MODEL_DIR+"/model_new.h5"
#If the specified file path is a model, get it and continue training
model=get_model_from_file_or_create(file_path)
#callbacks = get_callbacks(filepath=file_path, patience=5)
model.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=20,
          verbose=1,
          validation_data=(X_valid, y_valid))#,
          #callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 75, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 64)          36928     
__________

In [13]:
score = model.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

401/401 [==============================] - 4s 10ms/step
Test loss: 0.423774004874
Test accuracy: 0.862842893511


In [17]:
#pseudo labeling

def pseudo_labeling( y_train_cv):
    
    test = pd.read_json(TEST_PATH)
    X_test = prepare_data(test)
    
    #create labels for test data from existing model
    Y_test_pred = model.predict(X_test, batch_size=24)
    y_train_cv = y_train_cv.reshape(y_train_cv.shape[0], 1)

    #combine existing training data with test data
    x_combo = np.concatenate([X_train_cv, X_test])
    y_combo = np.concatenate([y_train_cv, Y_test_pred])

    #train model with pseudo data
    model.fit(x_combo, y_combo,
          verbose=1, batch_size=24, epochs=4, 
          validation_data=(X_valid, y_valid))

    #print score
    score = model.evaluate(X_valid, y_valid, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
pseudo_labeling( y_train_cv)

Train on 9627 samples, validate on 401 samples
Epoch 1/4
9627/9627 [==============================] - 298s 31ms/step - loss: 0.2525 - acc: 0.1326 - val_loss: 0.3976 - val_acc: 0.8329
Epoch 2/4
9627/9627 [==============================] - 295s 31ms/step - loss: 0.2286 - acc: 0.1336 - val_loss: 0.4259 - val_acc: 0.8479
Epoch 3/4
9627/9627 [==============================] - 296s 31ms/step - loss: 0.2209 - acc: 0.1341 - val_loss: 0.3686 - val_acc: 0.8479
Epoch 4/4
401/401 [==============================] - 4s 10ms/step
Test loss: 0.360267382682
Test accuracy: 0.870324189972


## Test and submission

In [18]:
def make_predictions_and_submit(test_data, file_path="sub.csv"):
    '''test_data - pandas dataframe'''
    
    #transform it in the right format for the model, applying data augmentation etc
    X_test = prepare_data(test_data)
    
    predicted_test=model.predict_proba(X_test)
    submission = pd.DataFrame()
    submission['id']=test_data['id']
    submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
    submission.to_csv(file_path, index=False)

In [19]:
#NOTE the test data is about 2GB. Uncomment if you are sure you want to execute this
#Load the test data.
test = pd.read_json(TEST_PATH)
make_predictions_and_submit(test)